In [1]:
import random

In [2]:
class State():
    def __init__(self, name, value, reciprocal, src):
        self._name = name
        self._value = value
        self._src = src
        self._reciprocal = reciprocal

    def get_name(self):
        return self._name
        
    def get_value(self):
        return self._value
    
    def get_src(self):
        return self._src
    
    def round_down(self, value=1):
        self._reciprocal -= value
        if self._reciprocal <= 0:
            return False
        return True
    
    def set_state(self, value, reciprocal, src):
        self._value = value
        self._src = src
        self._reciprocal = reciprocal

In [3]:
class Character():
    def __init__(self,
                 life,
                 attack,
                 defence,
                 speed,):
        self._life = life
        self._attack = attack
        self._defence = defence
        self._speed = speed
        self._states = []
        self._action_count = 0
        
    def set_target(self, target):
        self._target = target
        
    def round_start(self):
        self._action_count += 1
        if self._is_burn():
            self._life -= 7
            
        
    def action(self, target):
        for state in self._states:
            if state.get_name() == 'paralysis':
                return False
        for state in self._states:
            if state.get_name() == 'dominate':
                self._normal_attack(self)
                if self._life <= 0:
                    return False
        return True
            
    
    def _normal_attack(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        if self._is_confusion():
            self.damaged(attack, self)
        else:
            target.damaged(attack, self)
        
    def _active_skill(self, target):
        return
    
    def _passive_skill(self, target):
        return
    
    def damaged(self, value, target, true_damage = False):
        if true_damage:
            self._life -= value
            return True
        else:
            defence = self._defence
            for state in self._states:
                if state.get_name() == 'defence change':
                    defence += state.get_value()
            damage = value - defence
            if damage <0:
                damage = 0
            self._life -= damage
            if damage:
                return True
            return False
        
    def get_speed(self):
        speed = self._speed
        for state in self._states:
            if state.get_name() == 'speed change':
                speed += state.get_value()
        return speed
    
    def get_life(self):
        return self._life
    
    def set_state(self, name, value, reciprocal, src, independent=False, refresh=False):
        if independent:
            self._states.append(State(name, value, reciprocal, src))
        elif refresh:
            for state in self._states:
                if state.get_name() == name and state.get_src() == src:
                    state.set_state(state.get_value()+value, reciprocal, src)
        else:
            for state in self._states:
                if state.get_name() == name:
                    state.set_state(value, reciprocal, src)
                    return
            self._states.append(State(name, value, reciprocal, src))
            
    def round_end(self):
        new_state = []
        for state in self._states:
            if state.round_down():
                new_state.append(state)
            elif state.get_name() == 'explosion':
                self._life -= state.get_value() * 10
            elif state.get_name() == 'counter attack':
                self.damaged(20, self._target)
                self.set_state("paralysis", None, 1, self._target)
        self._states = new_state
    
    def _is_burn(self):
        for state in self._states:
            if state.get_name() == 'burn':
                return True
        return False
    
    def _is_confusion(self):
        for state in self._states:
            if state.get_name() == 'confusion':
                return True
        return False
    
    def _active_skill_forbidden(self):
        for state in self._states:
            if state.get_name() == 'charm':
                return True
        return False
    
    def _passive_skill_forbidden(self):
        for state in self._states:
            if state.get_name() == 'passive forbidden':
                return True
        return False

In [4]:
class Thelema(Character):
    def __init__(self,
                 life=100,
                 attack=19,
                 defence=9,
                 speed=23):
        super(Thelema, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 2 == 0:
            self._active_skill(target)
        self._normal_attack(target)
        self._passive_skill(target)
    
    def _active_skill(self, target):
        if not self._active_skill_forbidden():
            target.damaged(16, self)
        
    def _passive_skill(self, target):
        if target != self and random.random() < 0.2 and not self._passive_skill_forbidden():
            target.set_state('dominate', None, 2, self)

In [5]:
class Songque(Character):
    def __init__(self,
                 life=100,
                 attack=19,
                 defence=9,
                 speed=20):
        super(Songque, self).__init__(life, attack, defence, speed)
        self._shadow = 0
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0 and not self._active_skill_forbidden():
            self._active_skill(target)
        self._normal_attack(target)
        self._passive_skill(target)
        
    def _normal_attack(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        attack += 4*self._shadow
        
        if self._is_confusion():
            self.damaged(attack, self)
        else:
            target.damaged(attack, self)
        
    def _active_skill(self, target):
        target.damaged(15 + 4*self._shadow, self)
    
    def _passive_skill(self, target):
        if random.random() < 0.25 and self._shadow < 3 and not self._passive_skill_forbidden():
            self._shadow += 1
            target.set_state('defence change', -4, 99, self, independent=True)
            target.set_state('speed change', -2, 99, self, independent=True)

In [6]:
class Dreamseeker(Character):
    def __init__(self,
                 life=100,
                 attack=16,
                 defence=8,
                 speed=21):
        super(Dreamseeker, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        self._passive_skill(target)
        if self._action_count % 3 ==0 and not self._active_skill_forbidden():
            self._active_skill(target)
        else:
            self._normal_attack(target)
        
        
    def _active_skill(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        attack = 1.3*attack
        target.damaged(int(attack), self, true_damage=True)
    
    def _passive_skill(self, target):
        if random.random() < 0.7 and not self._passive_skill_forbidden():
            if self._life < 50:
                self.set_state('attack change', 30-self._attack, 1, self)
            else:
                self.set_state('attack change', 24-self._attack, 1, self)

In [7]:
class Senadina(Character):
    def __init__(self,
                 life=100,
                 attack=21,
                 defence=6,
                 speed=23):
        super(Senadina, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        self._normal_attack(target)
        self._passive_skill(target)
        if self._action_count % 3 == 0 and not self._active_skill_forbidden():
            self._active_skill(target)
            self._passive_skill(target)
        
        
    def _active_skill(self, target):
        self.set_state("lightning", None, 2, self)
        target.damaged(15, self)
        target.set_state("defence change", -5, 2, self)
    
    def _passive_skill(self, target):
        if not self._passive_skill_forbidden():
            for state in self._states:
                if state.get_name() == 'lightning':
                    effect_count = random.randint(1, 3)
                    for _ in range(effect_count):
                        target.damaged(3, self, true_damage=True)
                    if effect_count == 3:
                        target.set_state('paralysis', None, 2, self)

In [8]:
class Lantern(Character):
    def __init__(self,
                 life=100,
                 attack=15,
                 defence=7,
                 speed=21):
        super(Lantern, self).__init__(life, attack, defence, speed)
        self._mode = ['break', 'burst', 'rapid']
        self._mode_idx = 0
        self._dodge = False
        
    def round_start(self):
        super().round_start()
        if self._action_count % 4 == 0 and not self._passive_skill_forbidden():
            self._dodge = True
        else:
            self._dodge = False
    
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 2 == 0:
            self._active_skill(target)
        else:
            self._normal_attack(target)
            self._passive_skill(target)
            
    def _active_skill(self, target):
        if self._active_skill_forbidden():
            return
        self._mode_idx = (self._mode_idx+1)%3
        target.damaged(17, self)
        self.set_state("attack change", 3, 99, self, True)
        self._passive_skill(target)
        
    def _passive_skill(self, target):
        if not self._passive_skill_forbidden():
            if self._mode[self._mode_idx] == 'burst':
                target.set_state("defence change", -6, 2, self)
            if self._mode[self._mode_idx] == 'rapid':
                target.set_state("burn", None, 3, self)
            
    def damaged(self, value, target, true_damage=False):
        if self._dodge:
            self._dodge = False
            return False
        else:
            return super().damaged(value, target, true_damage)

In [9]:
class Seraphim(Character):
    def __init__(self,
                 life=100,
                 attack=20,
                 defence=8,
                 speed=15):
        super(Seraphim, self).__init__(life, attack, defence, speed)
        self._has_book = False
        self._book_life = 0
        self._dodge_rate = 0
        
    def round_start(self):
        super().round_start()
        if self._has_book and not self._passive_skill_forbidden():
            self._life += 10
            self._dodge_rate = 0.2
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0 and not self._active_skill_forbidden():
            self._active_skill(target)
        else:
            self._normal_attack(target)
            
    def _active_skill(self, target):
        self._has_book = True
        self._book_life = 15
        target.damaged(int(target.get_life()*0.15), self, true_damage=True)
        
    def damaged(self, value, target, true_damage=False):
        if random.random() > self._dodge_rate:
            if self._has_book:
                self._book_life -= value
                if self._book_life <= 0:
                    self._has_book = False
                return False
            else:
                return super().damaged(value, target, true_damage)
        return False
            
    def round_end(self):
        super().round_end()
        self._dodge_rate = 0

In [36]:
class Theresa(Character):
    def __init__(self,
                 life=100,
                 attack=21,
                 defence=8,
                 speed=24):
        super(Theresa, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 ==0:
            self._active_skill(target)
        else:
            self._normal_attack(target)
            
    def _normal_attack(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        if self._is_confusion():
            if self.damaged(attack, self) and not self._passive_skill_forbidden():
                if random.random() < 0.25:
                    target.set_state("passive forbidden", None, 2, self)
        else:
            if target.damaged(attack, self) and not self._passive_skill_forbidden():
                if random.random() < 0.25:
                    target.set_state("passive forbidden", None, 2, self)
    
    def _active_skill(self, target):
        if self._active_skill_forbidden():
            return
        if random.random() < 0.7:
            if target.damaged(30, self) and not self._passive_skill_forbidden():
                if random.random() < 0.25:
                    target.set_state("passive forbidden", None, 2, self)
        else:
            if target.damaged(1, self) and not self._passive_skill_forbidden():
                if random.random() < 0.25:
                    target.set_state("passive forbidden", None, 2, self)
                self._life += 18
            
    def set_state(self, name, value, reciprocal, src, independent=False, refresh=False):
        super().set_state(name, value, reciprocal, src, independent, refresh)
        if not self._passive_skill_forbidden():
            if name in ['attack change', 'defence change', 'speed change'] and value < 0:
                self._life += 10
            if name in ['paralysis', 'charm']:
                self._life += 10

In [11]:
class Helia(Character):
    def __init__(self,
                 life=100,
                 attack=18,
                 defence=8,
                 speed=28):
        super(Helia, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0:
            self._active_skill(target)
        else:
            self._normal_attack(target)
        
    def _active_skill(self, target):
        if self._active_skill_forbidden():
            return
        effect_count = random.randint(2,8)
        for _ in range(effect_count):
            target.damaged(12, self)
        self._passive_skill(target)
        
    def _passive_skill(self, target):
        if not self._passive_skill_forbidden():
            target.set_state('explosion', random.randint(1,3), 2, self)

In [12]:
class Coralie(Character):
    def __init__(self,
                 life=100,
                 attack=26,
                 defence=10,
                 speed=18):
        super(Coralie, self).__init__(life, attack, defence, speed)
        self._attack_count = 0
       
    def round_start(self):
        super().round_start()
        self._passive_skill(self._target)
    
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 2 == 0 and not self._active_skill_forbidden():
            self._active_skill(target)
        else:
            if not self._passive_skill_forbidden():
                if random.random() > 0.15:
                    self._normal_attack(target)
                    self._attack_count += 1
                    self._passive_skill(target)
            else:
                self._normal_attack(target)
                self._attack_count += 1
                self._passive_skill(target)
            
            
    def _active_skill(self, target):
        if not self._passive_skill_forbidden():
            if random.random() > 0.15:
                target.damaged(16, self)
                self._attack_count += 1
            if random.random() > 0.15 and random.random() < 0.3:
                target.damaged(25, self)
                self._attack_count += 1
        else:
            target.damaged(16, self)
            self._attack_count += 1
            if random.random() < 0.3:
                target.damaged(25, self)
                self._attack_count += 1
            
    def _passive_skill(self, target):
        if self._attack_count >= 3 and not self._passive_skill_forbidden():
            self._attack_count -= 3
            target.set_state('paralysis', None, 1, self)
            target.set_state('defence change', -5, 1, self)

In [13]:
class Vita(Character):
    def __init__(self,
                 life=100,
                 attack=22,
                 defence=10,
                 speed=25):
        super(Vita, self).__init__(life, attack, defence, speed)
        self.wings = False
        
    def round_start(self):
        super().round_start()
        if self.wings:
            self.wings=False
            self.set_state("paralysis", None, 1, self)
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0:
            self._active_skill(target)
        self._normal_attack(target)
        
    def _normal_attack(self, target):
        if self.wings:
            attack = self._attack + 7
        else:
            attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        if self._is_confusion():
            self.damaged(attack, self)
        else:
            target.damaged(attack, self)
        
    def _active_skill(self, target):
        if not self._active_skill_forbidden():
            self.wings = True
        
    def damaged(self, value, target, true_damage=False):
        if self.wings and not true_damage:
            super().damaged(value-3, target, true_damage)
        else:
            super().damaged(value, target, true_damage)
        if not self._passive_skill_forbidden() and random.random() < 0.25:
            target.set_state("charm", None, 2, self)
        if not self._passive_skill_forbidden() and self._life <= 0 and random.random() < 0.3:
            self._life = 20

In [14]:
class Mei(Character):
    def __init__(self,
                 life=100,
                 attack=18,
                 defence=9,
                 speed=24):
        super(Mei, self).__init__(life, attack, defence, speed)
        self._mode = ['fly', 'earth']
        self._mode_idx = 0
        self._dodge = False
        
    def round_start(self):
        super().round_start()
        if not self._passive_skill_forbidden() and self._mode[self._mode_idx] == 'fly' and random.random() < 0.25:
            self._dodge = True
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0 and not self._active_skill_forbidden():
            self._mode_idx = (self._mode_idx+1)%2
            if self._mode[self._mode_idx] == 'fly' and random.random() < 0.25:
                self._dodge = True
        self._normal_attack(target)
        
    def _normal_attack(self, target):
        if not self._passive_skill_forbidden():
            if self._mode[self._mode_idx] == 'fly':
                attack = self._attack + 8
            else:
                attack = self._attack - 2
        else:
            attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        if self._is_confusion():
            self.damaged(attack, self)
        else:
            target.damaged(attack, self)
        
    def damaged(self, value, target, true_damage = False):
        if self._dodge:
            target.set_state("counter attack", 20, 1, self)
            return False
        if true_damage:
            self._life -= value
            return True
        else:
            if not self._passive_skill_forbidden():
                if self._mode[self._mode_idx] == 'fly':
                    defence = self._defence - 2
            defence = self._defence
            for state in self._states:
                if state.get_name() == 'defence change':
                    defence += state.get_value()
            damage = value - defence
            if damage <0:
                damage = 0
            self._life -= damage
            if damage:
                return True
            return False
        
    def round_end(self):
        super().round_end()
        self._dodge = False
        
    def get_speed(self):
        if not self._passive_skill_forbidden() and self._mode[self._mode_idx] == 'earth':
            return super().get_speed()+2
        else:
            return super().get_speed()
        
    def set_state(self, name, value, reciprocal, src, independent=False, refresh=False):
        super().set_state(name, value, reciprocal, src, independent, refresh)
        if name in ['paralysis', 'charm']:
            if self._mode[self._mode_idx] == 'fly':
                self._mode_idx = 1
                if not self._passive_skill_forbidden():
                    for state in self._state:
                        if state.get_src() == self._target and state.get_name() in ['attack change', 'defence change', 'speed change']:
                            state.set_state(0, 0, self._target)

In [15]:
class Bronia(Character):
    def __init__(self,
                 life=100,
                 attack=18,
                 defence=6,
                 speed=20):
        super(Bronia, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0:
            self._active_skill(target)
        self._normal_attack(target)
        
    def _active_skill(self, target):
        if self._active_skill_forbidden():
            return
        for _ in range(5):
            if random.random() < 0.15 and not self._passive_skill_forbidden():
                target.damaged(16, self, true_damage=True)
            else:
                target.damaged(16, self)
        if random.random() < 0.2:
            target.set_state("confusion", None, 1 ,self)

In [16]:
def run(p1, p2):
    p1.set_target(p2)
    p2.set_target(p1)
    while p1.get_life() > 0 and p2.get_life() > 0:
        if p1.get_speed() > p2.get_speed():
            p1.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.action(p2)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.action(p1)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.round_end()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.round_end()
        else:
            p2.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.action(p1)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.action(p2)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.round_end()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.round_end()
    if p1.get_life() <= 0 and p2.get_life() <= 0:
        print(f"[Error], p1: {p1.get_life()}, p2: {p2.get_life()}")
    if p1.get_life() <= 0:
        return 0
    else:
        return 1

In [17]:
def show(p1, p2):
    p1.set_target(p2)
    p2.set_target(p1)
    while p1.get_life() > 0 and p2.get_life() > 0:
        if p1.get_speed() > p2.get_speed():
            p1.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.action(p2)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.action(p1)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.round_end()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.round_end()
        else:
            p2.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.round_start()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.action(p1)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.action(p2)
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.round_end()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.round_end()
        print("p1: {}, p2: {}".format(p1.get_life(), p2.get_life()))
    print("p1: {}, p2: {}".format(p1.get_life(), p2.get_life()))    
    if p1.get_life() <= 0 and p2.get_life() <= 0:
        print(f"[Error], p1: {p1.get_life()}, p2: {p2.get_life()}")
    if p1.get_life() <= 0:
        return 0
    else:
        return 1

In [39]:
count = 0
for i in range(100000):
    count += run(B(), Lantern())

p1: 85, p2: 90
p1: 70, p2: 80
p1: 46, p2: 22
p1: 31, p2: 12
p1: 16, p2: 2
p1: -8, p2: 2


0

# Day 3

In [21]:
count = 0
for i in range(100000):
    count += run(Songque(), Lantern())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.4)/100000,100000-count,(100000-count)*3.5/100000))

76328(1.0686) 23672(0.8285)


In [22]:
count = 0
for i in range(100000):
    count += run(Songque(), Seraphim())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.3)/100000,100000-count,(100000-count)*4.5/100000))

82345(1.0705) 17655(0.7945)


In [23]:
count = 0
for i in range(100000):
    count += run(Songque(), Seraphim())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.5)/100000,100000-count,(100000-count)*3.1/100000))

82190(1.2329) 17810(0.5521)


# Day 2

In [20]:
count = 0
for i in range(100000):
    count += run(Helia(), Vita())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.8)/100000,100000-count,(100000-count)*1.3/100000))

42484(1.6144) 57516(0.7477)


In [19]:
count = 0
for i in range(100000):
    count += run(Vita(), Bronia())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.4)/100000,100000-count,(100000-count)*3.2/100000))

84570(1.1840) 15430(0.4938)


In [20]:
count = 0
for i in range(100000):
    count += run(Helia(), Bronia())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.6)/100000,100000-count,(100000-count)*2.7/100000))

84125(1.3460) 15875(0.4286)


# Day 1

In [21]:
count = 0
for i in range(100000):
    count += run(Thelema(), Senadina())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.1)/100000,100000-count,(100000-count)*1.5/100000))

31102(0.9642) 68898(1.0335)


In [22]:
count = 0
for i in range(50000):
    count += run(Thelema(), Senadina())
for i in range(50000):
    count += 1 - run(Senadina(), Thelema())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.1)/100000,100000-count,(100000-count)*1.5/100000))

55269(1.7133) 44731(0.6710)


In [23]:
count = 0
for i in range(100000):
    count += 1 - run(Senadina(), Thelema())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.1)/100000,100000-count,(100000-count)*1.5/100000))

79959(2.4787) 20041(0.3006)


In [24]:
count = 0
for i in range(100000):
    count += run(Senadina(), Dreamseeker())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.4)/100000,100000-count,(100000-count)*3.7/100000))

73152(1.0241) 26848(0.9934)


In [25]:
count = 0
for i in range(100000):
    count += run(Thelema(), Dreamseeker())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.4)/100000,100000-count,(100000-count)*3.2/100000))

57249(0.8015) 42751(1.3680)
